## 多クラス分類の精度向上を目指そうの回

### ライブラリのインポート

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from statsbombpy import sb 
from mplsoccer import Pitch 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycaret
from pycaret.classification import *
from imblearn.over_sampling import SMOTENC

#import mlflow
#import dagshub

### 試合のデータ取得

In [ ]:
comp = sb.competitions()
#大会一覧取得(competition_id, season_idを知りたい)

In [ ]:
comp #Data Wranger(拡張機能で見ると楽かも？？)

In [4]:
#大会の情報取得
matches = sb.matches(competition_id=72, season_id= 107)


In [ ]:
matches #Data Wrangerで(以下略)

In [5]:
ids = list(matches['match_id'])

print(ids)

[3904629, 3906390, 3906389, 3904628, 3893806, 3893801, 3893797, 3893822, 3893814, 3893790, 3901832, 3902968, 3902967, 3901797, 3901796, 3902240, 3902239, 3893808, 3901833, 3893826, 3901736, 3901735, 3901734, 3901733, 3893823, 3893811, 3893812, 3893813, 3893810, 3893809, 3893807, 3893804, 3893788, 3893798, 3893789, 3893794, 3893792, 3893791, 3893787, 3893834, 3893833, 3893832, 3893831, 3893830, 3893829, 3893828, 3893827, 3893825, 3893824, 3893821, 3893820, 3893819, 3893818, 3893817, 3893816, 3893815, 3893805, 3893803, 3893802, 3893800, 3893799, 3893796, 3893795, 3893793]


In [8]:
#試合毎のイベントデータを一つのDFにまとめる関数
def match_concat(match_all):
    df_event = pd.DataFrame()
    for i in match_all:
        df_i = sb.events(match_id = i) #イベントデータ取得
        df_event = pd.concat([df_event,df_i], ignore_index= True) #くっつける
    return df_event

In [10]:
matches_WWC_2023 = match_concat(match_all = ids)

In [ ]:
matches_WWC_2023

In [31]:
event = matches_WWC_2023

#eventをpycaretに認識しやすい形　+　SMOTEで不均衡データのオーバーサンプリングを実施できるようにする
#DataFrameの複製
df2 = event.copy()

#データの抽出(ball_receipt_outcome)

df2 = df2[df2['type'] == 'Ball Receipt*']

df2['ball_receipt_outcome'] = df2['ball_receipt_outcome'].fillna('Complete')


# 結果の確認
#print(df2)

event['ball_receipt_outcome'] = df2['ball_receipt_outcome']

event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event.ball_recovery_recovery_failure = event.ball_recovery_recovery_failure.fillna(False)
event['block_deflection'] = event['block_deflection'].fillna(False)
event['clearance_aerial_won'] = event['clearance_aerial_won'].fillna(False)#.astype('boolean')
event['clearance_head'] = event['clearance_head'].fillna(False)#.astype('boolean')
event['clearance_right_foot'] = event['clearance_right_foot'].fillna(False)#.astype('boolean')
event['clearance_left_foot'] = event['clearance_left_foot'].fillna(False)#.astype('boolean')
event['clearance_body_part'] = event['clearance_body_part'].fillna('0').astype('category')
event['dribble_overrun'] = event['dribble_overrun'].fillna(False)
event['foul_committed_advantage'] = event['foul_committed_advantage'].fillna(False)#.astype('boolean')
event['foul_committed_card'] = event['foul_committed_card'].fillna('No Card')
event['foul_committed_card'] = event['foul_committed_card'].fillna('0').astype('category') 
event['foul_committed_offensive'] = event['foul_committed_offensive'].fillna(False)
event['foul_won_advantage'] = event['foul_won_advantage'].fillna(False)#.astype('boolean')
event['foul_won_defensive'] = event['foul_won_defensive'].fillna(False)
#goalkeepr_end_potition
df2 = event[event['type'] == 'Goal Keeper' ]

event_X_Y= df2['goalkeeper_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['goalkeeper_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['goalkeeper_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

#carry_end_location
df2 = event[event['type'] == 'Carry']

event_X_Y= df2['carry_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['carry_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['carry_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['carry_end_location_x'] = df2['carry_end_location_x']
event['carry_end_location_y'] = df2['carry_end_location_y']

#location
event_X_Y= event['location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['location_x'] = event_X_Ydf[0]

#y座標の代入
event['location_y'] = event_X_Ydf[1]

event['off_camera'] = event['off_camera'].fillna(False)
event['out'] = event['out'].fillna(False)
event['counterpress'] = event['counterpress'].fillna(False)
event['pass_aerial_won'] = event['pass_aerial_won'].fillna(False)#.astype('boolean')
event['pass_cross'] = event['pass_cross'].fillna(False)#.astype('boolean')
event['pass_cut_back'] = event['pass_cut_back'].fillna(False)#.astype('boolean')
event['pass_goal_assist'] = event['pass_goal_assist'].fillna(False)#.astype('boolean')
event['pass_inswinging'] = event['pass_inswinging'].fillna(False)#.astype('boolean')
event['pass_no_touch'] = event['pass_no_touch'].fillna(False)#.astype('boolean')
event['pass_outswinging'] = event['pass_outswinging'].fillna(False)#.astype('boolean')
event['pass_through_ball'] = event['pass_through_ball'].fillna(False)#.astype('boolean')
event['pass_shot_assist'] = event['pass_shot_assist'].fillna(False)#.astype('boolean')
event['pass_straight'] = event['pass_straight'].fillna(False)#.astype('boolean')
event['pass_switch'] = event['pass_switch'].fillna(False)


event_X_Y= event['pass_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['pass_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['pass_end_location_y'] = event_X_Ydf[1]


event['shot_aerial_won'] = event['shot_aerial_won'].fillna(False)#.astype('boolean')
event['shot_first_time'] = event['shot_first_time'].fillna(False)#.astype('boolean')
event['shot_one_on_one'] = event['shot_one_on_one'].fillna(False)
event['under_pressure'] = event['under_pressure'].fillna(False)

#shot_end_location
event_X_Y= event['shot_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#x座標を代入
event['shot_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['shot_end_location_y'] = event_X_Ydf[1]

#z座標の取得
event['shot_end_location_z'] = event_X_Ydf[2]

#元のdataに結合

#event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
#event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

event.dribble_outcome = event.dribble_outcome.fillna('0').astype('category')
event.duel_outcome = event.duel_outcome.fillna('0').astype('category')
event.duel_type =event.duel_type.fillna('0').astype('category')
event.duration = event.duration.fillna('0')#.astype('float')
event.foul_committed_type = event.foul_committed_type.fillna('0').astype('category')
event.goalkeeper_body_part = event.goalkeeper_body_part.fillna('0').astype('category')
event.goalkeeper_outcome = event.goalkeeper_outcome.fillna('0').astype('category')
event.goalkeeper_position = event.goalkeeper_position.fillna('0').astype('category')
event.goalkeeper_technique = event.goalkeeper_technique.fillna('0').astype('category')
event.goalkeeper_type = event.goalkeeper_type.fillna('0').astype('category')
event.interception_outcome = event.interception_outcome.fillna('0').astype('category')
event.pass_assisted_shot_id = event.pass_assisted_shot_id.fillna('0').astype('category')
event.pass_body_part = event.pass_body_part.fillna('0').astype('category')
event.pass_height = event.pass_height.fillna('0').astype('category')
event.pass_length = event.pass_length.fillna('0').astype('category')
event.pass_outcome = event.pass_outcome.fillna('0').astype('category')
event.pass_recipient = event.pass_recipient.fillna('0').astype('category')
event.pass_recipient_id = event.pass_recipient_id.fillna('0').astype('category')
event.pass_technique = event.pass_technique.fillna('0').astype('category')
event.pass_type = event.pass_type.fillna('0').astype('category')
event.player = event.player.fillna('No player').astype('category')
event.player_id = event.player_id.fillna('0').astype('int')
event.position = event.position.fillna('0').astype('category')
event.shot_body_part = event.shot_body_part.fillna('0').astype('category')
event.shot_key_pass_id = event.shot_key_pass_id.fillna('0').astype('category')
event.shot_outcome = event.shot_outcome.fillna('0').astype('category')
event.shot_statsbomb_xg = event.shot_statsbomb_xg.fillna('0.0').astype('float')
event.shot_technique = event.shot_technique.fillna('0').astype('category')
event.substitution_replacement = event.substitution_replacement.fillna('0').astype('category')
event.substitution_replacement_id = event.substitution_replacement_id.fillna('0').astype('int')
event.play_pattern = event.play_pattern.astype('category')
event.id = event.id.astype('category')
event.possession_team = event.possession_team.astype('category')
event.team = event.team.astype('category')


event = event.drop(['shot_type', 'substitution_outcome_id','substitution_outcome',
                    'related_events', 'tactics', 'shot_freeze_frame','shot_end_location',
                    'pass_end_location','injury_stoppage_in_chain',
                    'goalkeeper_end_location', 'carry_end_location','location','50_50'],
                    axis=1)


#event

In [33]:


event.bad_behaviour_card = event.bad_behaviour_card.fillna('0').astype('category')
event.dribble_nutmeg = event.dribble_nutmeg.fillna(False)
event.ball_recovery_offensive = event.ball_recovery_offensive.fillna(False)
event.clearance_other = event.clearance_other.fillna(False)
event.foul_committed_penalty = event.foul_committed_penalty.fillna(False)
event.miscontrol_aerial_won = event.miscontrol_aerial_won.fillna(False)
event.pass_deflected = event.pass_deflected.fillna(False)
event.shot_deflected = event.shot_deflected.fillna(False)
event.foul_won_penalty = event.foul_won_penalty.fillna(False)
event.block_offensive = event.block_offensive.fillna(False)
event.pass_miscommunication = event.pass_miscommunication.fillna(False)
event.goalkeeper_lost_in_play = event.goalkeeper_lost_in_play.fillna(False)
event.goalkeeper_shot_saved_to_post = event.goalkeeper_shot_saved_to_post.fillna(False)
event.shot_open_goal = event.shot_open_goal.fillna(False)
event.shot_saved_to_post = event.shot_saved_to_post.fillna(False)
event.goalkeeper_shot_saved_off_target = event.goalkeeper_shot_saved_off_target.fillna(False)
event.goalkeeper_success_in_play = event.goalkeeper_success_in_play.fillna(False)
event.shot_saved_off_target = event.shot_saved_off_target.fillna(False)
event.goalkeeper_punched_out = event.goalkeeper_punched_out.fillna(False)
event.block_save_block = event.block_save_block.fillna(False)
event.goalkeeper_penalty_saved_to_post = event.goalkeeper_penalty_saved_to_post.fillna(False)
event.dribble_no_touch = event.dribble_no_touch.fillna(False)
event.goalkeeper_success_out = event.goalkeeper_success_out.fillna(False)
event.shot_follows_dribble = event.shot_follows_dribble.fillna(False)
event.player_off_permanent = event.player_off_permanent.fillna(False)
event.shot_redirect = event.shot_redirect.fillna(False)



In [34]:
event.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226146 entries, 0 to 226145
Data columns (total 114 columns):
 #    Column                            Dtype   
---   ------                            -----   
 0    bad_behaviour_card                category
 1    ball_receipt_outcome              category
 2    ball_recovery_recovery_failure    bool    
 3    block_deflection                  bool    
 4    clearance_aerial_won              bool    
 5    clearance_body_part               category
 6    clearance_head                    bool    
 7    clearance_left_foot               bool    
 8    clearance_right_foot              bool    
 9    counterpress                      bool    
 10   dribble_nutmeg                    bool    
 11   dribble_outcome                   category
 12   duel_outcome                      category
 13   duel_type                         category
 14   duration                          object  
 15   foul_committed_advantage          bool    
 16   

In [ ]:
event

In [45]:
event_fi.value_counts('type')

type
Others     117371
Pass        59837
Dribble     48938
dtype: int64

In [44]:
event_fi = event.drop(['pass_angle', 'carry_end_location_x', 'carry_end_location_y', 'goalkeeper_end_location_x', 
                       'goalkeeper_end_location_y', 'pass_end_location_x', 'pass_end_location_y', 
                       'shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z'], axis=1)

event_fi = event_fi.replace({'type':{'Carry':'Dribble'}})

#replace({'age': [24, 18], 'point': 70}, 100))

event_fi = event_fi.replace({'type':
                            ['Ball Receipt*', 'Pressure','Ball Recovery', 'Duel', 'Goal Keeper', 'Shot', 'Block', 'Miscontrol',  'Clearance', 'Dispossessed',
                             'Interception', 'Foul Committed', 'Dribbled Past', 'Foul Won', 'Substitution','Half Start', 'Half End', 'Injury Stoppage',
                             'Player Off','Player On','Referee Ball-Drop','Starting XI', 'Tactical Shift', 'Shield', 'Error','50/50','Offside', 'Bad Behaviour',
                             'Own Goal Against', 'Own Goal For']},
                            'Others')

#event_fi
#event_fi.info()

le = LabelEncoder()
le.fit(event_fi['type'])

event_fi['type_num'] = le.transform(event_fi['type'])

event_fi

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,block_save_block,goalkeeper_penalty_saved_to_post,dribble_no_touch,goalkeeper_success_out,shot_follows_dribble,player_off_permanent,shot_redirect,location_x,location_y,type_num
0,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
1,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
2,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
3,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
4,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226141,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
226142,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
226143,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1
226144,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,1


# pycaret用オブジェクトとデータ分割、学習

In [46]:

event_fi_py = event_fi.drop('timestamp', axis=1)

In [47]:
ml_data = event_fi_py.sample(frac=0.8, random_state=5).reset_index(drop=True)
ml_data_unseen = event_fi_py.drop(ml_data.index).reset_index(drop=True)


In [ ]:
#実行はしない(Data Wrenger)

ml_data_unseen

In [48]:
exp_trial = setup(data = ml_data, target= 'type_num' ,
                  ignore_features = ["type","id", "pass_recipient","pass_recipient_id", "pass_aerial_won", "pass_assisted_shot_id",
                                     "pass_body_part", "pass_height", "pass_inswinging", "pass_length", "pass_type", "player_id", "team_id"], 
                  fix_imbalance = True, use_gpu=True,
                  
                  session_id= 1)
#log_experiment = "dagshub", experiment_name = 'trial_classification_2', log_plots = True, log_data =True,

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Description,Value
0,Session id,1
1,Target,type_num
2,Target type,Multiclass
3,Original data shape,"(180917, 104)"
4,Transformed data shape,"(251772, 217)"
5,Transformed train set shape,"(197496, 217)"
6,Transformed test set shape,"(54276, 217)"
7,Ignore features,13
8,Numeric features,11
9,Categorical features,27


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


In [49]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7373,0.9171,0.7373,0.8097,0.6897,0.5678,0.6108,25.2650
lightgbm,Light Gradient Boosting Machine,0.7283,0.9229,0.7283,0.7945,0.6962,0.5616,0.6081,14.8110
ada,Ada Boost Classifier,0.7148,0.0000,0.7148,0.7638,0.6717,0.5311,0.5689,39.7520
gbc,Gradient Boosting Classifier,0.7148,0.0000,0.7148,0.8291,0.6782,0.5436,0.6037,271.5030
lda,Linear Discriminant Analysis,0.6970,0.0000,0.6970,0.7560,0.7068,0.5329,0.5508,22.9200
ridge,Ridge Classifier,0.6924,0.0000,0.6924,0.7612,0.7030,0.5283,0.5497,14.4370
et,Extra Trees Classifier,0.6721,0.8960,0.6721,0.6511,0.6450,0.4244,0.4418,31.1940
dt,Decision Tree Classifier,0.6653,0.7513,0.6653,0.6780,0.6528,0.4553,0.4703,14.7230
qda,Quadratic Discriminant Analysis,0.6300,0.0000,0.6300,0.8037,0.6431,0.4550,0.5235,17.5530
xgboost,Extreme Gradient Boosting,0.6260,0.9098,0.6260,0.7408,0.5492,0.2711,0.3802,19.1780


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [50]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7413,0.9224,0.7413,0.7951,0.6900,0.5678,0.6035
1,0.7315,0.9146,0.7315,0.8145,0.6881,0.5619,0.6085
2,0.7386,0.9149,0.7386,0.8083,0.6886,0.5677,0.6086
3,0.7380,0.9166,0.7380,0.8114,0.6887,0.5699,0.6154
4,0.7327,0.9143,0.7327,0.8069,0.6846,0.5604,0.6035
5,0.7435,0.9211,0.7435,0.8099,0.6962,0.5769,0.6179
6,0.7352,0.9163,0.7352,0.8121,0.6887,0.5660,0.6111
7,0.7404,0.9174,0.7404,0.8033,0.6899,0.5709,0.6122
8,0.7370,0.9168,0.7370,0.8135,0.6910,0.5685,0.6125


In [54]:
tuned_rf = tune_model(rf, optimize='Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [55]:
tuned_rf_prec = tune_model(rf, optimize='Prec.')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [56]:
tuned_rf_F1 = tune_model(rf, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [57]:
tuned_rf_Kappa = tune_model(rf, optimize='Kappa')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [58]:
tuned_rf_MCC = tune_model(rf, optimize='MCC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [59]:
tuned_rf_ac = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
